## obain absorption energies

### calculate molecule energies

In [15]:
%%writefile energies_molecule.py
import pickle
from ase.build import molecule
from ase.visualize import view
from ase.optimize import BFGS
from ase.parallel import parprint, paropen
from ase import Atoms
from gpaw import GPAW, PW

energies = {}    
for name in ['H', 'OH', 'Li', 'Mg']:
    if name == 'Mg':
        system = Atoms('Mg',
                        positions=[[5, 5, 5]],
                        cell=[10, 10, 10],
                        )
    else:
        system = molecule(name)
        system.center(vacuum=10)

    calc = GPAW(mode=PW(500),
                xc='RPBE',
                #kpts={'size': (5, 5, 1), 'gamma': True},
                kpts={'density': 3, 'gamma': True},  
                #poissonsolver={'dipolelayer': 'xy'},
                txt='out-molecule-'+name+'.out',
                )
    if name == 'H' or name == 'Li': 
        calc.set(hund=True)

    system.set_calculator(calc)
    relax = BFGS(system, trajectory='relax--molecule-'+name+'.traj', logfile='log-molecule-'+name+'.log')
    relax.run(fmax=0.1)
    energy = system.get_potential_energy()
    energies[name] = energy
    
with paropen('energies_molecule.pckl', 'wb') as f:
    pickle.dump(energies, f)

Writing energies_molecule.py


In [16]:
!bsub.py -t 3 -p 20 energies_molecule.py

Job <8466896> is submitted to queue <hpc>.


### print all energies

In [18]:
#%%writefile all_energies.py
import pickle
from ase.parallel import paropen

print("========================================energies_slab====================================================")
energies = {}
with paropen('energies_slab.pckl', 'rb') as f:
    energies = pickle.load(f)
print(energies, 'length:', len(energies), '\n')

print("=======================================energies_molecule================================================")
with paropen('energies_molecule.pckl', 'rb') as f:
    energies_molecule = pickle.load(f)
energies.update(energies_molecule)
print(energies_molecule, 'length:', len(energies_molecule), '\n')

print("========================================energies_part2==================================================")
with paropen('energies_part2.pckl', 'rb') as f:
    energies2 = pickle.load(f)
energies.update(energies2)
print(energies2, 'length:', len(energies2), '\n')

print("====================================energies_compressive_part3==========================================")
with paropen('energies_compressive_part3.pckl', 'rb') as f:
    energies3 = pickle.load(f)
energies.update(energies3)
print(energies3, 'length:', len(energies3), '\n')

print("====================================energies_tensile_part3.pckl==========================================")
with paropen('energies_tensile_part3.pckl', 'rb') as f:
    energies3_2 = pickle.load(f)
energies.update(energies3_2)
print(energies3_2, 'length:', len(energies3_2), '\n')

print("======================================energies_tube_slab================================================")
with paropen('energies_tube_slab.pckl', 'rb') as f:
    energies_tube_slab = pickle.load(f)
energies.update(energies_tube_slab)
print(energies_tube_slab, 'length:', len(energies_tube_slab), '\n')

print("======================================energies_tubes_part4==============================================")
with paropen('energies_tubes_part4.pckl', 'rb') as f:
    energies4 = pickle.load(f)
energies.update(energies4)
print(energies4, 'length:', len(energies4), '\n')

# print("========================================all energies====================================================")
# print(energies, 'length:', len(energies), '\n')

# with paropen('all_energies.pckl', 'wb') as f:
#     pickle.dump(energies, f)


========================================energies_slab====================================================
{'BiCl2': -9.221547347249004, 'BiS2': -12.05987966223659, 'BiSCl': -11.716367625749408} length: 3 

=======================================energies_molecule================================================
{'H': -1.0764131717375824, 'OH': -7.186320422731457, 'Li': -0.2586393069787914, 'Mg': -0.09885299776753707} length: 4 

========================================energies_part2==================================================
{'BiCl2-H-top': -87.98501516065777, 'BiCl2-H-bridge': -89.21375414959753, 'BiCl2-H-hollow': -89.09319164306903, 'BiCl2-OH-top': -93.04468579693774, 'BiCl2-OH-bridge': -92.95692592681606, 'BiCl2-OH-hollow': -92.33215380842712, 'BiCl2-Li-top': -87.90306039070168, 'BiCl2-Li-bridge': -91.03654017036094, 'BiCl2-Li-hollow': -88.91092364668445, 'BiCl2-Mg-top': -85.77841944176068, 'BiCl2-Mg-bridge': -91.48185146739378, 'BiCl2-Mg-hollow': -88.46834537822637, 'BiS2-H-to

###  Adsorption energy of H, OH, Li, and Mg on BiCl2, BiS2, BiSCl

In [4]:
import pandas as pd
dataframe = []
adsorption_energies = {}
for stru, ener in energies2.items():
#     print(stru, ener)
    separate = stru.split('-')
#     print(separate[0], '|', separate[1])
#     print(energies[separate[0]], '|', energies[separate[1]])
    adsorption_energies[stru] = ener - 9 * energies[separate[0]] - energies[separate[1]]
#     print(adsorption_energies[stru])
    dataframe.append({'Structures': stru, 'Adsorption energies': adsorption_energies[stru]})
df = pd.DataFrame(dataframe)
df = df[['Structures', 'Adsorption energies']]

In [6]:
df
df.sort_values(by=['Adsorption energies'])

,Structures,Adsorption energies
10,BiCl2-Mg-bridge,-8.389072
7,BiCl2-Li-bridge,-7.783975
8,BiCl2-Li-hollow,-5.658358
11,BiCl2-Mg-hollow,-5.375566
1,BiCl2-H-bridge,-5.143415
2,BiCl2-H-hollow,-5.022852
6,BiCl2-Li-top,-4.650495
23,BiS2-Mg-hollow,-4.194107
22,BiS2-Mg-bridge,-4.182802
20,BiS2-Li-hollow,-3.955007


### Adsorption energy of H, OH, Li, and Mg on BiCl2, BiS2 and BiSCl after adding compressive strain

In [9]:
import pandas as pd
dataframe = []
adsorption_energies = {}
for stru, ener in energies3.items():
#     print(stru, ener)
    separate = stru.split('-')
#     print(separate[0], '|', separate[1])
#     print(energies[separate[0]], '|', energies[separate[1]])
    adsorption_energies[stru] = ener - 9 * energies[separate[0]] - energies[separate[1]]
#     print(adsorption_energies[stru])
    dataframe.append({'Structures': stru, 'Adsorption energies': adsorption_energies[stru]})
df = pd.DataFrame(dataframe)
df = df[['Structures', 'Adsorption energies']]

In [23]:
df
df.sort_values(by=['Adsorption energies'])

,Structures,Adsorption energies
10,BiCl2-Mg-bridge,-8.547643
7,BiCl2-Li-bridge,-8.015553
8,BiCl2-Li-hollow,-5.956920
2,BiCl2-H-hollow,-5.868361
11,BiCl2-Mg-hollow,-5.859877
1,BiCl2-H-bridge,-5.805058
6,BiCl2-Li-top,-5.042657
3,BiCl2-OH-top,-5.019536
0,BiCl2-H-top,-4.443587
23,BiS2-Mg-hollow,-4.278749


### Adsorption energy of H, OH, Li, and Mg on BiCl2, BiS2 and BiSCl after adding tensile strain

In [19]:
import pandas as pd
dataframe = []
adsorption_energies = {}
for stru, ener in energies3_2.items():
#     print(stru, ener)
    separate = stru.split('-')
#     print(separate[0], '|', separate[1])
#     print(energies[separate[0]], '|', energies[separate[1]])
    adsorption_energies[stru] = ener - 9 * energies[separate[0]] - energies[separate[1]]
#     print(adsorption_energies[stru])
    dataframe.append({'Structures': stru, 'Adsorption energies': adsorption_energies[stru]})
df = pd.DataFrame(dataframe)
df = df[['Structures', 'Adsorption energies']]

In [22]:
df
df.sort_values(by=['Adsorption energies'])

,Structures,Adsorption energies
10,BiCl2-Mg-bridge,-8.547643
7,BiCl2-Li-bridge,-8.015553
8,BiCl2-Li-hollow,-5.956920
2,BiCl2-H-hollow,-5.868361
11,BiCl2-Mg-hollow,-5.859877
1,BiCl2-H-bridge,-5.805058
6,BiCl2-Li-top,-5.042657
3,BiCl2-OH-top,-5.019536
0,BiCl2-H-top,-4.443587
23,BiS2-Mg-hollow,-4.278749


### Adsorption energy of H, OH, Li, and Mg on BiSCl tube

In [15]:
import pandas as pd
dataframe = []
adsorption_energies = {}
for stru, ener in energies4.items():
#     print(stru, ener)
    separate = stru.split('-')
#     print(separate[0], '|', separate[1])
#     print(energies[separate[0]], '|', energies[separate[1]])
    adsorption_energies[stru] = ener - 2 * energies[separate[0]] - energies[separate[1]]
#     print(adsorption_energies[stru])
    dataframe.append({'Structures': stru, 'Adsorption energies': adsorption_energies[stru]})
df = pd.DataFrame(dataframe)
df = df[['Structures', 'Adsorption energies']]

In [17]:
df
df.sort_values(by=['Adsorption energies'])

,Structures,Adsorption energies
7,BiSCl_1D-Li-bridge,-2.349398
8,BiSCl_1D-Li-hollow,-2.290684
20,BiSCl_1D-Li-hollow-inner,-2.211896
19,BiSCl_1D-Li-bridge-inner,-2.124492
18,BiSCl_1D-Li-top-inner,-1.785108
6,BiSCl_1D-Li-top,-1.638452
14,BiSCl_1D-H-hollow-inner,-1.194575
11,BiSCl_1D-Mg-hollow,-1.166399
12,BiSCl_1D-H-top-inner,-1.074613
0,BiSCl_1D-H-top,-1.059141
